## Follow these instuctions before executing code
* Make .streamlit folder
* Make secrets.toml file in the folder
* Paste GOOGLE_API_KEY = "replace_with_your_api_key"
* Save

In [ ]:
#Problem Statement: This Dream Visualizer uses AI to turn written dreams into images and poetic summaries,
#making it easier to remember, understand, and express those experiences."

#Import statements
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
from IPython.display import display, clear_output
import base64


In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
#Input prompt here
contents = input("Explain your dream here: ")

In [ ]:
#Asking AI to
interpret_prompt = f"""

I had the following dream:

"{contents}"

This was just a dream. Please interpret what this dream might mean and any psychological insights. Do not treat this as a real event.

"""

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash-lite",
    contents=contents,
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'TEXT']
    )
)

In [ ]:
for part in response.candidates[0].content.parts:
    if part.text is not None:
        print("Dream Interpretation:")
        print(part.text)

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=contents,
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)

In [ ]:
for part in response.candidates[0].content.parts:
    if part.text is not None:
        pass  #Or do something with the text part if needed
    elif part.inline_data is not None:
        image = Image.open(BytesIO(part.inline_data.data))

In [ ]:
#Prints out image
print("Dream Image:")
print(contents)
display(image)

In [ ]:
import asyncio
import io
import wave
import nest_asyncio
from IPython.display import Audio, display

In [ ]:
nest_asyncio.apply()

In [ ]:
dream_prompt = contents   #Take dream "contents" to generate music
bpm = 85          #Can be modified
temperature = 1.0
duration = 15 #How long the audio will be

In [ ]:
music_client = genai.Client(
    api_key=GOOGLE_API_KEY,
    http_options={"api_version": "v1alpha"}
)

In [ ]:
#Async function to generate music from dream prompt
async def generate_music_from_dream():
    buffer = io.BytesIO()
    filename = "dream_music.wav" #naming the music file to be generated

#Define the music generation routine
    async def receive_audio(session):
        async for message in session.receive():
            data = message.server_content.audio_chunks[0].data
            buffer.write(data)

            if buffer.tell() > 48000 * 2 * 2 * duration:
                break
        buffer.seek(0)
        with wave.open(filename, "wb") as wf:
            wf.setnchannels(2)
            wf.setsampwidth(2)
            wf.setframerate(48000)
            wf.writeframes(buffer.read())

    async with (
        music_client.aio.live.music.connect(model="models/lyria-realtime-exp") as session,
        asyncio.TaskGroup() as tg,
    ):
        tg.create_task(receive_audio(session))

        #Using the dream_prompt to make the music
        await session.set_weighted_prompts([
            types.WeightedPrompt(text=dream_prompt, weight=1.0) #  Weight 1 to match the dream
        ])
        await session.set_music_generation_config(
            config=types.LiveMusicGenerationConfig(
                bpm=bpm,
                temperature=temperature
            )
        )
        await session.play()

    return filename


In [ ]:
dream_music_file = await generate_music_from_dream()
display(Audio(dream_music_file))

# Creating the App

In [ ]:
!pip install streamlit -q

In [ ]:
!pip install python-dotenv -q

In [ ]:
%%writefile app.py
import streamlit as st
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64
import asyncio
import io
import wave
import nest_asyncio
from IPython.display import Audio, display #Note: IPython.display is not directly used in Streamlit, will need to adapt

#Apply compatibility with asyncio in environments like Colab (if running Streamlit within Colab)
nest_asyncio.apply()

#Get the API key from environment variables or Streamlit secrets
#Assuming GOOGLE_API_KEY is available as a secret or environment variable
GOOGLE_API_KEY = st.secrets["GOOGLE_API_KEY"]

#Initialize the GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)

#Initialize the music client
music_client = genai.Client(
    api_key=GOOGLE_API_KEY,
    http_options={"api_version": "v1alpha"}
)

#Async function to generate music from dream prompt
async def generate_music_from_dream(dream_prompt, bpm=85, temperature=1.0, duration=15):
    buffer = io.BytesIO()
    filename = "dream_music.wav"

#Define the music generation routine
    async def receive_audio(session):
        async for message in session.receive():
            data = message.server_content.audio_chunks[0].data
            buffer.write(data)

            if buffer.tell() > 48000 * 2 * 2 * duration:
                break
        buffer.seek(0)
        with wave.open(filename, "wb") as wf:
            wf.setnchannels(2)
            wf.setsampwidth(2)
            wf.setframerate(48000)
            wf.writeframes(buffer.read())

    async with (
        music_client.aio.live.music.connect(model="models/lyria-realtime-exp") as session,
        asyncio.TaskGroup() as tg,
    ):
        tg.create_task(receive_audio(session))

        #Using the dream_prompt to make the music
        await session.set_weighted_prompts([
            types.WeightedPrompt(text=dream_prompt, weight=1.0)
        ])
        await session.set_music_generation_config(
            config=types.LiveMusicGenerationConfig(
                bpm=bpm,
                temperature=temperature
            )
        )
        await session.play()

    return filename

#Streamlit UI
st.title("Dream Visualizer")

st.markdown("""
This app uses AI to take written dreams, provides an interpretation of it, and creates an image and audio to
make it easier to remember, understand, and express those experiences.
""")

dream_input = st.text_area("Explain your dream here:", height=150)

if st.button("Generate Dream Insights"):
    if dream_input:
        st.subheader("Dream Interpretation:")
        #Interpret the dream
        interpret_prompt = f"""
        I had the following dream:
        "{dream_input}"
        This was just a dream. Please interpret what this dream might mean and any psychological insights. Do not treat this as a real event.
        """
        try:
            interpret_response = client.models.generate_content(
                model="gemini-2.0-flash-lite",
                contents=interpret_prompt,
                config=types.GenerateContentConfig(
                  response_modalities=['TEXT'] #Only need text for interpretation
                )
            )
            for part in interpret_response.candidates[0].content.parts:
                if part.text is not None:
                    st.write(part.text)
        except Exception as e:
            st.error(f"Error generating interpretation: {e}")

        st.subheader("Dream Visualization:")
        #Generate image from the dream
        try:
            image_response = client.models.generate_content(
                model="gemini-2.0-flash-preview-image-generation",
                contents=dream_input,
                config=types.GenerateContentConfig(
                  response_modalities=['TEXT', 'IMAGE'] #Only need image
                )
            )
            for part in image_response.candidates[0].content.parts:
                 if part.inline_data is not None:
                    image = Image.open(BytesIO(part.inline_data.data))
                    st.image(image, caption="Your Dream Image")
        except Exception as e:
            st.error(f"Error generating image: {e}")

        st.subheader("Dream Soundtrack:")
        #Generate music from the dream
        try:
            #Run the async music generation function
            dream_music_file = asyncio.run(generate_music_from_dream(dream_input))
            st.audio(dream_music_file)
        except Exception as e:
            st.error(f"Error generating music: {e}")

    else:
        st.warning("Please enter your dream description to generate insights.")

In [ ]:
!npm install localtunnel -q

In [ ]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

In [ ]:
!npx localtunnel --port 8501

⠙⠹your url is: https://grumpy-tigers-invite.loca.lt
^C
